In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train_df = pd.read_pickle('../input/mini-model-data/train_df_s.pkl')
test_df = pd.read_pickle('../input/mini-model-data/test_df_s.pkl')

train_identity = pd.read_pickle('../input/mini-model-data/train_identity.pkl')
test_identity = pd.read_pickle('../input/mini-model-data/test_identity.pkl')

train = train_df.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_df.merge(test_identity, how='left', left_index=True, right_index=True)

train_y = train_df['isFraud'].copy()

# print(train_df.shape)
# print(test_df.shape)
# print(train_identity.shape)
# print(test_identity.shape)
print(train.shape)
print(test.shape)

del train_df, test_df, train_identity, test_identity

(11628, 433)
(11418, 433)


In [3]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987038,0,87172,25.0,S,5463,399.0,150.0,american express,137.0,credit,330.0,87.0,NaN,98.0,NaN,gmail.com,1.0,2.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,93.0,1.0,0.0,96.0,1.0,542.0,542.0,NaN,NaN,NaN,537.0,4.0,29.000000,0.000000,520.0,NaN,NaN,NaN,NaN,541.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,6.0,1.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,145.0,25.0,0.0,145.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,125.0,25.0,0.0,0.0,25.0,30.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0,

In [4]:
target_encode_book = [
    'addr1', 'C1', 'C2', 'C3', 'C4', 'C8', 'C10', 'C11', 'C13', 'C14', 
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'D9',
    'device_name', 'DeviceType',
    'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_19',
    'id_20', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
    'id_30_device', 'id_30_version', 'lastest_browser', 
    'id_32', 'id_33', 'id_33_0', 'id_33_1', 'id_33_area', 'id_33_ratio','id_35', 'id_36', 'id_37', 'id_38',
    'R_emaildomain', 'cents'
]
# CV: C13, C14

In [5]:
def fe_dt(df):
    
    df['DT'] = df['TransactionDT'].apply(
        lambda x: (START_DATE + datetime.timedelta(seconds=x)))  # tmp    
    df['DT_month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_week_year'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_day_year'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    
    for col in ['D1', 'D2', 'D3', 'D6', 'D7', 'D8','D10', 'D13','D14', 'D15']:
        # day map
        day_temp_dict = df.groupby('DT_day_year')[col].agg(['mean']).rename(columns={'mean':col + '_day_mean'})
        day_temp_dict = day_temp_dict[col + '_day_mean'].to_dict()
        df[col + '_day_mean'] = df['DT_day_year'].map(day_temp_dict)
        df[col + '_day_mean'] = df[col] - df[col + '_day_mean']
        
        # week map
        week_temp_dict = df.groupby('DT_week_year')[col].agg(['mean']).rename(columns={'mean':col + '_week_mean'})
        week_temp_dict = week_temp_dict[col + '_week_mean'].to_dict()
        df[col + '_week_mean'] = df['DT_week_year'].map(week_temp_dict)
        df[col + '_week_mean'] = df[col] - df[col + '_week_mean']

    df = df.drop(['DT'],axis=1)
    
    return df

In [6]:
def fe_uid(train_df, test_df):
    
    train_df['uid'] = train_df['card1'].astype(str)+'_'+train_df['card2'].astype(str)
    test_df['uid'] = test_df['card1'].astype(str)+'_'+test_df['card2'].astype(str)

    train_df['uid2'] = train_df['uid'].astype(str)+'_'+train_df['card3'].astype(str)+'_'+train_df['card5'].astype(str)
    test_df['uid2'] = test_df['uid'].astype(str)+'_'+test_df['card3'].astype(str)+'_'+test_df['card5'].astype(str)

    train_df['uid3'] = train_df['uid2'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'+train_df['addr2'].astype(str)
    test_df['uid3'] = test_df['uid2'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'+test_df['addr2'].astype(str)

    # Check if the Transaction Amount is common or not (we can use freq encoding here)
    # In our dialog with a model we are telling to trust or not to these values   
    train_df['TransactionAmt_check'] = np.where(train_df['TransactionAmt'].isin(test_df['TransactionAmt']), 1, 0)
    test_df['TransactionAmt_check']  = np.where(test_df['TransactionAmt'].isin(train_df['TransactionAmt']), 1, 0)

    i_cols = ['card1','card2','card3','card5','uid','uid2','uid3']

    for col in i_cols:
        for agg_type in ['mean','std']:
            new_col_name = col+'_TransactionAmt_'+agg_type
            temp_df = pd.concat([train_df[[col, 'TransactionAmt']], test_df[[col,'TransactionAmt']]])
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            train_df[new_col_name] = train_df[col].map(temp_df)
            test_df[new_col_name]  = test_df[col].map(temp_df)

    train_df['TransactionAmt'] = np.log1p(train_df['TransactionAmt'])
    test_df['TransactionAmt'] = np.log1p(test_df['TransactionAmt'])   
    
    train_df = train_df.drop(['uid','uid2','uid3'],axis=1)
    test_df = test_df.drop(['uid','uid2','uid3'],axis=1)
    
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)
    
    return train_df, test_df

In [7]:
# feature engineering

def fe(train, test):

    drop_na_cols = []
    for col in train.columns:
        if (len(train[col].dropna().unique()) <= 1):
            drop_na_cols.append(col)
    print(len(drop_na_cols), ' features are removed...')
    # remove all nan columns
    train = train.drop(drop_na_cols, axis=1)
    test = test.drop(draop_na_cols, axis=1)
    
    # user id
    train, test = fe_uid(train, test)

    ignore_cols = ['R_emaildomain', 'DeviceInfo', 'id_30', 'id_31', 'id_33']
    for col in train.columns:
        if (train[col].dtype == 'object' or test[col].dtype == 'object') and col not in ignore_cols:
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[col].values) + list(test[col].values))
            train[col] = lbl.transform(list(train[col].values))
            test[col] = lbl.transform(list(test[col].values))
    
    for df in [train, test]:
        # device info
        df = setDevice(df)
        
        # id_30
        df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
        df['id_30_device'] = df['id_30'].apply(
            lambda x: ''.join([i for i in x if i.isalpha()]))
        df['id_30_version'] = df['id_30'].apply(
            lambda x: ''.join([i for i in x if i.isnumeric()]))
        
        # id_31
        df = setBrowser(df)
        
        # nan encoding
        for col in train.columns:
            if(train[col].isnull().sum() / len(train) > 0.2):
                df[col + '_isnull'] = df[col].isnull() + 0
        
        # email map
        for item in ['R_emaildomain']:
            df[item] = df[item].map(emails)
            
        # id_33
        df['id_33'] = df['id_33'].fillna('0x0')
        df['id_33_0'] = df['id_33'].apply(lambda x: x.split('x')[0]).astype(int)
        df['id_33_1'] = df['id_33'].apply(lambda x: x.split('x')[1]).astype(int)
        df['id_33'] = np.where(df['id_33']=='0x0', np.nan, df['id_33'])
        df['id_33_area'] = df['id_33_0'] * df['id_33_1']
        df['id_33_ratio'] = df['id_33_0'] / df['id_33_1']
        
        # cents
        df['cents'] = df['TransactionAmt'].apply(lambda x: x - int(x))
        

    train['cents'] = train['TransactionAmt'].apply(lambda x: x - int(x))
    test['cents'] = test['TransactionAmt'].apply(lambda x: x - int(x))
    
    for col in target_encode_book:
        train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)
        
        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
        train[col+'_fq_enc'] = train[col].map(fq_encode)
        test[col+'_fq_enc']  = test[col].map(fq_encode)

    un_corr_cols = []
    for col in train.columns:
        if (train[col].dtype != 'object'):
            if(abs(pearsonr(train[col].fillna(-999), train_y)[1]) > 0.05):
                un_corr_cols.append(col)
            print(col, pearsonr(train[col].fillna(-999), train_y))

    train = fe_dt(train)
    test = fe_dt(test)
    
    train = train.drop(target_encode_book + un_corr_cols, axis=1)
    test = test.drop(target_encode_book + un_corr_cols, axis=1)
    
    return train, test, drop_na_cols + un_corr_cols


train, test, drop_cols = fe(train, test)

76  features are removed...
isFraud (1.0, 0.0)
TransactionDT (0.029003805978637453, 0.001760630108036341)
TransactionAmt (0.005166433663987953, 0.5774885378794739)
card1 (-0.05524194100866575, 2.5096480199261694e-09)
card2 (-0.07573776923223491, 2.887957078068026e-16)
card3 (-0.0076893066489547015, 0.4070559187244568)
card4 (-0.012988671731964548, 0.16135807875529348)
card5 (0.02837273123490984, 0.0022147777056446788)
card6 (0.010297377553767719, 0.2668657749715858)
addr1 (0.00658061283309591, 0.4779894597861666)
addr2 (0.033862132533060384, 0.0002600873383476244)
dist2 (-0.01095928623186352, 0.23733135098348765)
C1 (0.25883690370401996, 2.4379272192743363e-177)
C2 (0.12081692590833212, 4.60023414419825e-39)
C3 (-0.0639512559174746, 5.11336357884836e-12)
C4 (0.11188045690255678, 1.0415605160868074e-33)
C6 (0.07372076349979353, 1.7270330039492255e-15)
C8 (0.02890306181361364, 0.001826855464481884)
C10 (-0.02758588667280082, 0.002930641404723335)
C11 (0.13415139338927717, 7.8061061142083

V224 (0.07653362325071422, 1.407514283489333e-16)
V225 (0.07669820570913281, 1.2119984751454495e-16)
V226 (0.07601009124355802, 2.2601954482773734e-16)
V227 (-0.002771520543999386, 0.7650694645750478)
V228 (0.07688998545678394, 1.0177589371718212e-16)
V229 (0.07713359345040112, 8.147389898546716e-17)
V230 (0.07702675620504298, 8.983224466829726e-17)
V231 (0.07238048524254206, 5.521179218784755e-15)
V232 (0.07164413478610962, 1.0362568895623574e-14)
V233 (0.07170060057186388, 9.876349522680083e-15)
V234 (-0.041901531778454834, 6.187596105344578e-06)
V235 (0.07667186773090216, 1.2413816211826305e-16)
V236 (0.0760501075542859, 2.1800819589034135e-16)
V237 (0.07622255920506235, 1.8656745280221e-16)
V238 (-0.00388774399666732, 0.6750813658184691)
V239 (-0.0036091528518518217, 0.6971681871016218)
V240 (0.07638250638033897, 1.6142416437046965e-16)
V242 (0.07678145469436665, 1.123560403018978e-16)
V243 (0.07683584275476713, 1.0692508185200791e-16)
V244 (0.07678884841062918, 1.1160206420948123e

card4_target_code (0.0740332081539418, 1.3132041587531824e-15)
card4_fq_enc (-0.021248239888523812, 0.021947332463815885)
card5_target_code (0.11858767096666316, 1.0906883877113665e-37)
card5_fq_enc (-0.008800226056779572, 0.34268524339710227)
card6_target_code (0.009299605705273956, 0.3159959009895961)
card6_fq_enc (-0.010071890911257915, 0.2774811793930534)
D9_target_code (0.08276911814363579, 3.9043556910882165e-19)
D9_fq_enc (-0.030286652792645863, 0.0010896083742764147)
device_name_target_code (0.1341438306065832, 7.900872335126694e-48)
device_name_fq_enc (-0.06609405701309802, 9.73736013964331e-13)
DeviceType_target_code (0.12051922542921809, 7.045274837824186e-39)
DeviceType_fq_enc (-0.11913352720349396, 5.051788764709502e-38)
id_12_target_code (0.052388521205837, 1.5813416442024637e-08)
id_12_fq_enc (0.050974034459368954, 3.804858106384064e-08)
id_13_target_code (0.10834626136082172, 1.0482829207230426e-31)
id_13_fq_enc (-0.07285881877410777, 3.655442819657183e-15)
id_14_target

In [8]:
train.head()

,isFraud,TransactionDT,addr2,C6,D1,D2,D3,D6,D7,D10,D13,D14,D15,V14,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V31,V32,V55,V56,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V71,V72,V77,V78,V80,V81,V84,V85,V86,V87,V88,V89,V92,V93,V95,V96,V97,V99,V100,V101,V102,V103,V104,V105,V106,V108,V109,V110,V111,V114,V115,V116,V120,V123,V124,V125,V126,V127,V128,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V160,V162,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V207,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V223,V224,V225,V226,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V240,V242,V243,V244,V246,V247,V248,V249,V252,V253,V254,V257,V258,V260,V261,V262,V263,V264,V265,V266,V268,V269,V270,V271,V272,V273,V274,V275,V279,V280,V281,V282,V283,V285,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V306,V307,V308,V310,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V326,V327,V328,V329,V330,V331,V332,V333,V335,V336,V337,V338,V339,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_30,id_31,id_34,DeviceInfo,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,D3_isnull,D6_isnull,D7_isnull,D8_isnull,D9_isnull,D10_isnull,D13_isnull,D14_isnull,D15_isnull,V14_isnull,V17_isnull,V18_isnull,V19_isnull,V20_isnull,V21_isnull,V22_isnull,V23_isnull,V24_isnull,V25_isnull,V26_isnull,V27_isnull,V28_isnull,V31_isnull,V32_isnull,V55_isnull,V56_isnull,V59_isnull,V60_isnull,V61_isnull,V62_isnull,V63_isnull,V64_isnull,V65_isnull,V66_isnull,V67_isnull,V68_isnull,V71_isnull,V72_isnull,V77_isnull,V78_isnull,V80_isnull,V81_isnull,V84_isnull,V85_isnull,V86_isnull,V87_isnull,V88_isnull,V89_isnull,V92_isnull,V93_isnull,id_03_isnull,id_04_isnull,id_09_isnull,id_10_isnull,id_13_isnull,addr1_target_code,addr1_fq_enc,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C3_target_code,C3_fq_enc,C4_target_code,C4_fq_enc,C8_target_code,C8_fq_enc,C10_target_code,C10_fq_enc,C11_target_code,C11_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card4_target_code,card4_fq_enc,card5_target_code,D9_target_code,D9_fq_enc,device_name_target_code,device_name_fq_enc,DeviceType_target_code,DeviceType_fq_enc,id_12_target_code,id_12_fq_enc,id_13_target_code,id_13_fq_enc,id_14_target_code,id_14_fq_enc,id_15_target_code,id_15_fq_enc,id_16_target_code,id_16_fq_enc,id_17_target_code,id_17_fq_enc,id_19_target_code,id_19_fq_enc,id_20_target_code,id_23_target_code,id_26_target_code,id_28_target_code,id_29_target_code,id_30_device_target_code,id_30_device_fq_enc,id_30_version_target_code,id_30_version_fq_enc,id_32_target_code,id_32_fq_enc,id_33_target_code,id_33_fq_enc,id_33_0_target_code,id_33_0_fq_enc,id_33_1_target_code,id_33_1_fq_enc,id_33_area_target_code,id_33_area_fq_enc,id_33_ratio_target_code,id_33_ratio_fq_enc,id_35_target_code,id_35_fq_enc,id_36_target_code,id_37_target_code,id_37_fq_enc,id_38_target_code,R_emaildomain_target_code,R_emaildomain_fq_enc,cents_target_code,cents_fq_enc,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,D1_day_mean,D1_week_mean,D2_day_mean,D2_week_mean,D3_day_mean,D3_week_mean,D6_day_mean,D6_week_mean,D7_day_mean,D7_week_mean,D8_day_mean,D8_week_mean,D10_day_mean,D10_week_mean,D13_day_mean,D13_week_mean,D14_day_mean,D14_week_mean,D15_day_mean,D15_week_mean
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [9]:
# col = 'V19'
# train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)

# plt.scatter(train[train.isFraud==0]['TransactionDT'], train[train.isFraud==0][col+'_target_code'], s=8)
# plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col+'_target_code'], s=8)

In [10]:
train.to_pickle('../input/mini-model-features/train_features_s.pkl')
test.to_pickle('../input/mini-model-features/test_features_s.pkl')

# POINT_SIZE = 2
# for col in train.columns:
#     fig = plt.figure(figsize=(12,8))

#     plt.scatter(train[train['DT_month'] == 12]['TransactionDT'], train[train['DT_month'] == 12][col], s=POINT_SIZE, c='b')     
#     plt.scatter(train[train['DT_month'] == 13]['TransactionDT'], train[train['DT_month'] == 13][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 14]['TransactionDT'], train[train['DT_month'] == 14][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 15]['TransactionDT'], train[train['DT_month'] == 15][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 16]['TransactionDT'], train[train['DT_month'] == 16][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 17]['TransactionDT'], train[train['DT_month'] == 17][col], s=POINT_SIZE, c='k')

#     plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col], s=POINT_SIZE * 3, c='r')
    
#     plt.scatter(test['TransactionDT'], test[col], s=POINT_SIZE, c='g')
#     plt.title(col)
#     plt.grid()
# #     plt.show()
    
#     fig.savefig("../eda/mini-models/S/S_" + str(col) + ".png")
#     plt.cla()
#     plt.clf()
#     plt.close(fig)
